## Aqui fazemos a primeira etapa do code, onde é feito o processamento dos dados baixados da ação correspondente, direto do site da Fundamentus. São explicados todos os passos do processamento. 

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import yfinance as yf
from fancyimpute import KNN


pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Using TensorFlow backend.


# DADOS FUNDAMENTALISTAS - FUNDAMENTUS

Ao fazer o download do site da Fundamentus, os dados vem em formato excel, então pegamos e convertemos facilmente para cvs.

Aqui trazemos o csv da ação para o notebook. Ele necessita do ".T" para que ele faça uma transposição, pois os atributos estavam nas linhas e as amostras nas colunas, assim invertemos isso, ficando no formato convencional utilizado.

Após, dropamos uma coluna inútil.

A coluna que mostra as datas, que estão separadas por trimestre, fica com um título NaN e está na primeira posição das colunas, assim precisamos atribuir o devido nome a coluna.

Assim, criamos uma coluna chamada "Data Trimeste", ao que lhe é atribuida as datas, e na sequência deletamos a coluna com o títlo NaN, através do iloc que engloba apenas colunas que tenham título "notnull"(naõ nulos).


A coluna "Data Trimestre" fica posicionada como última coluna do dataset, para ficar mais claro, apagamos, colocamos em uma variavel auxiliar, e a inserimos na primeira posição das colunas.

In [2]:
########################## COMEÇA O CÓDIGO QUE EXTRAI OS DADOS FUNDAMENTALISTAS ##########################

fundamental_data = pd.read_csv("goau4.csv",index_col=0, header=None).T
fundamental_data.drop(columns = 'XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data', inplace = True)


fundamental_data['Data Trimestre'] = fundamental_data.iloc[0:,0]
fundamental_data = fundamental_data.iloc[:, fundamental_data.columns.notnull()]

data = fundamental_data['Data Trimestre']
fundamental_data.drop(labels=['Data Trimestre'], axis=1,inplace = True)
fundamental_data.insert(0, 'Data Trimestre', data)


fundamental_data.head()

,Data Trimestre,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2019,51.170.656,16.937.624,1.750.949,390.607,3.732.421,9.531.907,0,960.663,0,571.076,6.530.975,0,0,0,0,0,0,3.627.519,0,63.126,1.695.371,16.258.302,9.748.383,0,51.170.656,7.964.919,482.729,3.832.827,546.534,2.122.757,0,0,919.099,60.973,0,16.824.918,12.799.975,5.198,1.939.291,61.015,0,2.019.439,0,0,16.816.360,9.564.456,7.965.507,-69.861,0,728.495,245.699,694.616,0,0,0
2,31/03/2019,52.190.061,17.611.692,1.941.977,590.403,3.976.755,9.377.329,0,905.645,0,819.582,6.893.503,0,0,0,0,0,3.787.600,0,74.160,3.031.743,1.379.597,16.392.413,9.912.856,0,52.190.061,9.543.682,406.224,4.066.779,911.088,3.184.456,0,0,910.033,65.102,0,16.388.055,12.054.153,5.285,2.154.186,59.101,0,2.115.330,0,0,16.738.614,9.519.709,7.963.819,-69.861,0,728.495,152.495,744.761,0,0,0
3,31/12/2018,51.344.417,17.563.118,2.891.217,459.470,3.201.656,9.167.689,0,1.031.581,0,811.505,6.917.916,0,0,0,0,0,0,3.874.054,0,27.939,1.367.802,15.547.094,9.948.486,0,51.344.417,9.156.188,590.110,4.119.074,765.013,2.473.789,0,0,1.147.626,60.576,0,16.815.756,13.081.776,1.350,1.413.478,118.368,0,2.200.785,0,0,16.027.297,9.345.175,7.960.908,-69.861,0,728.550,0,725.577,0,0,0
4,30/09/2018,56.180.777,22.227.653,2.795.394,679.994,4.193.568,9.124.052,0,941.142,0,4.493.503,6.353.695,0,0,0,0,0,0,3.610.574,0,41.506,1.472.356,15.807.410,10.319.663,0,56.180.777,9.817.638,609.593,4.149.752,614.163,2.841.377,0,0,1.563.664,39.089,0,20.153.991,16.015.383,1.395,1.328.951,266.132,0,2.542.132,0,0,16.571.375,9.637.772,7.956.661,-69.861,0,174.017,572.546,1.004.408,0,0,0
5,30/06/2018,54.517.367,20.704.301,2.638.013,303.724,4.100.884,8.479.855,0,909.635,0,4.272.191,6.252.401,0,0,0,0,0,0,3.511.040,0,42.690,1.454.877,16.037.499,10.068.290,0,54.517.367,9.155.147,509.455,4.204.288,528.291,2.730.270,0,0,1.152.978,29.865,0,20.240.859,16.044.958,1.344,1.608.288,155.660,0,2.430.611,0,0,15.932.185,9.189.175,7.955.263,-69.861,0,174.017,358.485,771.272,0,0,0


Aqui é o processo de manipulação das datas, muito importante na execução do code.

Primeiro utilizamos pd.to_datetime, função própria do Pandas para que ele entenda o formato e de que se trata de informações sobre data.

Subdividimos em colunas distintas, informações sobre "Dia", "Mês" e "Ano", logo a coluna "Data Trimestre" já não nos era mais útil.

Assim, novamente colocamos as três novas colunas em váriaveis auxiliares, para posiciona-las nas primeiras colunas do dataset.

In [3]:

fundamental_data['Data Trimestre'] = pd.to_datetime(fundamental_data['Data Trimestre'].str.strip(), format='%d/%m/%Y')


fundamental_data["Day"] = fundamental_data['Data Trimestre'].map(lambda x: x.day)
fundamental_data["Month"] = fundamental_data['Data Trimestre'].map(lambda x: x.month)
fundamental_data["Year"] = fundamental_data['Data Trimestre'].map(lambda x: x.year)

fundamental_data.drop(columns = 'Data Trimestre', inplace = True)

year = fundamental_data['Year']
fundamental_data.drop(labels=['Year'], axis=1,inplace = True)
fundamental_data.insert(0, 'Year', year)

month = fundamental_data['Month']
fundamental_data.drop(labels=['Month'], axis=1,inplace = True)
fundamental_data.insert(1, 'Month', month)

day = fundamental_data['Day']
fundamental_data.drop(labels=['Day'], axis=1,inplace = True)
fundamental_data.insert(2, 'Day', day)


fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,2019,6,30,51.170.656,16.937.624,1.750.949,390.607,3.732.421,9.531.907,0,960.663,0,571.076,6.530.975,0,0,0,0,0,0,3.627.519,0,63.126,1.695.371,16.258.302,9.748.383,0,51.170.656,7.964.919,482.729,3.832.827,546.534,2.122.757,0,0,919.099,60.973,0,16.824.918,12.799.975,5.198,1.939.291,61.015,0,2.019.439,0,0,16.816.360,9.564.456,7.965.507,-69.861,0,728.495,245.699,694.616,0,0,0
2,2019,3,31,52.190.061,17.611.692,1.941.977,590.403,3.976.755,9.377.329,0,905.645,0,819.582,6.893.503,0,0,0,0,0,3.787.600,0,74.160,3.031.743,1.379.597,16.392.413,9.912.856,0,52.190.061,9.543.682,406.224,4.066.779,911.088,3.184.456,0,0,910.033,65.102,0,16.388.055,12.054.153,5.285,2.154.186,59.101,0,2.115.330,0,0,16.738.614,9.519.709,7.963.819,-69.861,0,728.495,152.495,744.761,0,0,0
3,2018,12,31,51.344.417,17.563.118,2.891.217,459.470,3.201.656,9.167.689,0,1.031.581,0,811.505,6.917.916,0,0,0,0,0,0,3.874.054,0,27.939,1.367.802,15.547.094,9.948.486,0,51.344.417,9.156.188,590.110,4.119.074,765.013,2.473.789,0,0,1.147.626,60.576,0,16.815.756,13.081.776,1.350,1.413.478,118.368,0,2.200.785,0,0,16.027.297,9.345.175,7.960.908,-69.861,0,728.550,0,725.577,0,0,0
4,2018,9,30,56.180.777,22.227.653,2.795.394,679.994,4.193.568,9.124.052,0,941.142,0,4.493.503,6.353.695,0,0,0,0,0,0,3.610.574,0,41.506,1.472.356,15.807.410,10.319.663,0,56.180.777,9.817.638,609.593,4.149.752,614.163,2.841.377,0,0,1.563.664,39.089,0,20.153.991,16.015.383,1.395,1.328.951,266.132,0,2.542.132,0,0,16.571.375,9.637.772,7.956.661,-69.861,0,174.017,572.546,1.004.408,0,0,0
5,2018,6,30,54.517.367,20.704.301,2.638.013,303.724,4.100.884,8.479.855,0,909.635,0,4.272.191,6.252.401,0,0,0,0,0,0,3.511.040,0,42.690,1.454.877,16.037.499,10.068.290,0,54.517.367,9.155.147,509.455,4.204.288,528.291,2.730.270,0,0,1.152.978,29.865,0,20.240.859,16.044.958,1.344,1.608.288,155.660,0,2.430.611,0,0,15.932.185,9.189.175,7.955.263,-69.861,0,174.017,358.485,771.272,0,0,0


Essa é uma parte bem crítica, resetamos o índice(para que ele fique com a contagem correta de 0 até o total de linhas). E você deve estar se perguntando: por quê pegar o dataset a partir da 4 posição ?

Esse dataset está em formato descrescente com base nas datas, então, la no final do projeto, quando formos comparar os algortimos de classificação com a série temporal, para validação temos que comparar com dados que já ocorreram, par ter uma noção do desempenho.

Lembrando que se pegassemos de 0:, nesse caso, ele ia predizer dados a partir do 3T de 2018 até o 3T de 2019, sendo que o TCC estava sendo feito nessa época, então não tinhamos os dados reais necessarios para comparação, então por fins logisticos, reduzimos a ánalise um ano antes, para ao final do trabalho, quando compararmos os fundamentus e os algortimos de classificação com as séries temporais e a LSTM tenhamos tranquilidade para comparar com dados que ocorreram de fato para se ter uma boa base para inferir desempenho e tirar conclusões.

In [4]:
fundamental_data.reset_index(drop=True, inplace=True)
fundamental_data = fundamental_data[0:42]
fundamental_data.reset_index(drop=True, inplace=True)

fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2019,6,30,51.170.656,16.937.624,1.750.949,390.607,3.732.421,9.531.907,0,960.663,0,571.076,6.530.975,0,0,0,0,0,0,3.627.519,0,63.126,1.695.371,16.258.302,9.748.383,0,51.170.656,7.964.919,482.729,3.832.827,546.534,2.122.757,0,0,919.099,60.973,0,16.824.918,12.799.975,5.198,1.939.291,61.015,0,2.019.439,0,0,16.816.360,9.564.456,7.965.507,-69.861,0,728.495,245.699,694.616,0,0,0
1,2019,3,31,52.190.061,17.611.692,1.941.977,590.403,3.976.755,9.377.329,0,905.645,0,819.582,6.893.503,0,0,0,0,0,3.787.600,0,74.160,3.031.743,1.379.597,16.392.413,9.912.856,0,52.190.061,9.543.682,406.224,4.066.779,911.088,3.184.456,0,0,910.033,65.102,0,16.388.055,12.054.153,5.285,2.154.186,59.101,0,2.115.330,0,0,16.738.614,9.519.709,7.963.819,-69.861,0,728.495,152.495,744.761,0,0,0
2,2018,12,31,51.344.417,17.563.118,2.891.217,459.470,3.201.656,9.167.689,0,1.031.581,0,811.505,6.917.916,0,0,0,0,0,0,3.874.054,0,27.939,1.367.802,15.547.094,9.948.486,0,51.344.417,9.156.188,590.110,4.119.074,765.013,2.473.789,0,0,1.147.626,60.576,0,16.815.756,13.081.776,1.350,1.413.478,118.368,0,2.200.785,0,0,16.027.297,9.345.175,7.960.908,-69.861,0,728.550,0,725.577,0,0,0
3,2018,9,30,56.180.777,22.227.653,2.795.394,679.994,4.193.568,9.124.052,0,941.142,0,4.493.503,6.353.695,0,0,0,0,0,0,3.610.574,0,41.506,1.472.356,15.807.410,10.319.663,0,56.180.777,9.817.638,609.593,4.149.752,614.163,2.841.377,0,0,1.563.664,39.089,0,20.153.991,16.015.383,1.395,1.328.951,266.132,0,2.542.132,0,0,16.571.375,9.637.772,7.956.661,-69.861,0,174.017,572.546,1.004.408,0,0,0
4,2018,6,30,54.517.367,20.704.301,2.638.013,303.724,4.100.884,8.479.855,0,909.635,0,4.272.191,6.252.401,0,0,0,0,0,0,3.511.040,0,42.690,1.454.877,16.037.499,10.068.290,0,54.517.367,9.155.147,509.455,4.204.288,528.291,2.730.270,0,0,1.152.978,29.865,0,20.240.859,16.044.958,1.344,1.608.288,155.660,0,2.430.611,0,0,15.932.185,9.189.175,7.955.263,-69.861,0,174.017,358.485,771.272,0,0,0


Para fins didáticos, criei uma coluna "Quarter" que indica qual trimestre corresponde aquela linha, e a posicionei na primeira posição das colunas.

In [5]:
fundamental_data['Quarter'] = '2Q2019','1Q2019', '4Q2018', '3Q2018', '2Q2018', '1Q2018', '4Q2017', '3Q2017', '2Q2017', '1Q2017', '4Q2016', '3Q2016', '2Q2016', '1Q2016', '4Q2015', '3Q2015', '2Q2015', '1Q2015','4Q2014', '3Q2014', '2Q2014', '1Q2014', '4Q2013', '3Q2013', '2Q2013', '1Q2013', '4Q2012', '3Q2012', '2Q2012', '1Q2012', '4Q2011', '3Q2011', '2Q2011', '1Q2011', '4Q2010', '3Q2010', '2Q2010', '1Q2010', '4Q2009', '3Q2009', '2Q2009', '1Q2009'
quarter = fundamental_data['Quarter']
fundamental_data.drop(labels=['Quarter'], axis=1, inplace=True)
fundamental_data.insert(0, 'Quarter', quarter)

fundamental_data.head()

,Quarter,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,30,51.170.656,16.937.624,1.750.949,390.607,3.732.421,9.531.907,0,960.663,0,571.076,6.530.975,0,0,0,0,0,0,3.627.519,0,63.126,1.695.371,16.258.302,9.748.383,0,51.170.656,7.964.919,482.729,3.832.827,546.534,2.122.757,0,0,919.099,60.973,0,16.824.918,12.799.975,5.198,1.939.291,61.015,0,2.019.439,0,0,16.816.360,9.564.456,7.965.507,-69.861,0,728.495,245.699,694.616,0,0,0
1,1Q2019,2019,3,31,52.190.061,17.611.692,1.941.977,590.403,3.976.755,9.377.329,0,905.645,0,819.582,6.893.503,0,0,0,0,0,3.787.600,0,74.160,3.031.743,1.379.597,16.392.413,9.912.856,0,52.190.061,9.543.682,406.224,4.066.779,911.088,3.184.456,0,0,910.033,65.102,0,16.388.055,12.054.153,5.285,2.154.186,59.101,0,2.115.330,0,0,16.738.614,9.519.709,7.963.819,-69.861,0,728.495,152.495,744.761,0,0,0
2,4Q2018,2018,12,31,51.344.417,17.563.118,2.891.217,459.470,3.201.656,9.167.689,0,1.031.581,0,811.505,6.917.916,0,0,0,0,0,0,3.874.054,0,27.939,1.367.802,15.547.094,9.948.486,0,51.344.417,9.156.188,590.110,4.119.074,765.013,2.473.789,0,0,1.147.626,60.576,0,16.815.756,13.081.776,1.350,1.413.478,118.368,0,2.200.785,0,0,16.027.297,9.345.175,7.960.908,-69.861,0,728.550,0,725.577,0,0,0
3,3Q2018,2018,9,30,56.180.777,22.227.653,2.795.394,679.994,4.193.568,9.124.052,0,941.142,0,4.493.503,6.353.695,0,0,0,0,0,0,3.610.574,0,41.506,1.472.356,15.807.410,10.319.663,0,56.180.777,9.817.638,609.593,4.149.752,614.163,2.841.377,0,0,1.563.664,39.089,0,20.153.991,16.015.383,1.395,1.328.951,266.132,0,2.542.132,0,0,16.571.375,9.637.772,7.956.661,-69.861,0,174.017,572.546,1.004.408,0,0,0
4,2Q2018,2018,6,30,54.517.367,20.704.301,2.638.013,303.724,4.100.884,8.479.855,0,909.635,0,4.272.191,6.252.401,0,0,0,0,0,0,3.511.040,0,42.690,1.454.877,16.037.499,10.068.290,0,54.517.367,9.155.147,509.455,4.204.288,528.291,2.730.270,0,0,1.152.978,29.865,0,20.240.859,16.044.958,1.344,1.608.288,155.660,0,2.430.611,0,0,15.932.185,9.189.175,7.955.263,-69.861,0,174.017,358.485,771.272,0,0,0


Renomeei a coluna "Month" para "Cluster_Month", pois como se trata de trimestre, ele agrupa ali três meses correspondentes áquele trimestre, então sabemos que colunas onde tem Cluster_Month 3, englona na verdade, os meses 1,2 e 3. E assim por diante com os demais.

A coluna dia não nos é mais útil aqui, então tiramos a coluna.

In [6]:
fundamental_data.rename(columns={'Month' : 'Cluster_Month'},inplace = True)
fundamental_data.drop('Day', axis=1, inplace=True)

fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,51.170.656,16.937.624,1.750.949,390.607,3.732.421,9.531.907,0,960.663,0,571.076,6.530.975,0,0,0,0,0,0,3.627.519,0,63.126,1.695.371,16.258.302,9.748.383,0,51.170.656,7.964.919,482.729,3.832.827,546.534,2.122.757,0,0,919.099,60.973,0,16.824.918,12.799.975,5.198,1.939.291,61.015,0,2.019.439,0,0,16.816.360,9.564.456,7.965.507,-69.861,0,728.495,245.699,694.616,0,0,0
1,1Q2019,2019,3,52.190.061,17.611.692,1.941.977,590.403,3.976.755,9.377.329,0,905.645,0,819.582,6.893.503,0,0,0,0,0,3.787.600,0,74.160,3.031.743,1.379.597,16.392.413,9.912.856,0,52.190.061,9.543.682,406.224,4.066.779,911.088,3.184.456,0,0,910.033,65.102,0,16.388.055,12.054.153,5.285,2.154.186,59.101,0,2.115.330,0,0,16.738.614,9.519.709,7.963.819,-69.861,0,728.495,152.495,744.761,0,0,0
2,4Q2018,2018,12,51.344.417,17.563.118,2.891.217,459.470,3.201.656,9.167.689,0,1.031.581,0,811.505,6.917.916,0,0,0,0,0,0,3.874.054,0,27.939,1.367.802,15.547.094,9.948.486,0,51.344.417,9.156.188,590.110,4.119.074,765.013,2.473.789,0,0,1.147.626,60.576,0,16.815.756,13.081.776,1.350,1.413.478,118.368,0,2.200.785,0,0,16.027.297,9.345.175,7.960.908,-69.861,0,728.550,0,725.577,0,0,0
3,3Q2018,2018,9,56.180.777,22.227.653,2.795.394,679.994,4.193.568,9.124.052,0,941.142,0,4.493.503,6.353.695,0,0,0,0,0,0,3.610.574,0,41.506,1.472.356,15.807.410,10.319.663,0,56.180.777,9.817.638,609.593,4.149.752,614.163,2.841.377,0,0,1.563.664,39.089,0,20.153.991,16.015.383,1.395,1.328.951,266.132,0,2.542.132,0,0,16.571.375,9.637.772,7.956.661,-69.861,0,174.017,572.546,1.004.408,0,0,0
4,2Q2018,2018,6,54.517.367,20.704.301,2.638.013,303.724,4.100.884,8.479.855,0,909.635,0,4.272.191,6.252.401,0,0,0,0,0,0,3.511.040,0,42.690,1.454.877,16.037.499,10.068.290,0,54.517.367,9.155.147,509.455,4.204.288,528.291,2.730.270,0,0,1.152.978,29.865,0,20.240.859,16.044.958,1.344,1.608.288,155.660,0,2.430.611,0,0,15.932.185,9.189.175,7.955.263,-69.861,0,174.017,358.485,771.272,0,0,0


Correção de processamento, os dados vieram com "pontos" o que inviabiliza a manipulação númerica que o pandas compreende, assim tivemos que remover os pontos e atribuir o formato float, anteriormente veio com formato string.

Assim podemos ver a disposição dos números em relação a tabela gerada pela célula abaixo e a que tinhamos antes.

In [7]:
for key in ['Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC', 'Ativos Biológicos AC','Tributos a Recuperar AC','Despesas Antecipadas AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo ','Aplicações Financeiras Avaliadas a Valor Justo ACLP','Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP','Contas a Receber ACLP','Estoques ACLP','Ativos Biológicos ACLP','Tributos Diferidos ACLP','Despesas Antecipadas ACLP','Créditos com Partes Relacionadas ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Diferido','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Passivos com Partes Relacionadas PC','Dividendos e JCP a Pagar PC','Outros PC','Provisões PC', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC', 'Passivo Não Circulante','Empréstimos e Financiamentos PNC','Passivos com Partes Relacionadas PNC','Outros PNC','Tributos Diferidos PNC','Adiantamento para Futuro Aumento Capital PNC','Provisões PNC','Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC','Lucros e Receitas a Apropriar PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Reavaliação','Reservas de Lucros','Lucros/Prejuízos Acumulados','Ajustes de Avaliação Patrimonial','Ajustes Acumulados de Conversão','Outros Resultados Abrangentes','Adiantamento para Futuro Aumento Capital']:  
    fundamental_data[key] = fundamental_data[key].map(lambda x: str(x).replace(".", ""))
    fundamental_data[key] = fundamental_data[key].astype(float)


fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,51170656.0,16937624.0,1750949.0,390607.0,3732421.0,9531907.0,0.0,960663.0,0.0,571076.0,6530975.0,0.0,0.0,0.0,0.0,0.0,0.0,3627519.0,0.0,63126.0,1695371.0,16258302.0,9748383.0,0.0,51170656.0,7964919.0,482729.0,3832827.0,546534.0,2122757.0,0.0,0.0,919099.0,60973.0,0.0,16824918.0,12799975.0,5198.0,1939291.0,61015.0,0.0,2019439.0,0.0,0.0,16816360.0,9564456.0,7965507.0,-69861.0,0.0,728495.0,245699.0,694616.0,0.0,0.0,0.0
1,1Q2019,2019,3,52190061.0,17611692.0,1941977.0,590403.0,3976755.0,9377329.0,0.0,905645.0,0.0,819582.0,6893503.0,0.0,0.0,0.0,0.0,0.0,3787600.0,0.0,74160.0,3031743.0,1379597.0,16392413.0,9912856.0,0.0,52190061.0,9543682.0,406224.0,4066779.0,911088.0,3184456.0,0.0,0.0,910033.0,65102.0,0.0,16388055.0,12054153.0,5285.0,2154186.0,59101.0,0.0,2115330.0,0.0,0.0,16738614.0,9519709.0,7963819.0,-69861.0,0.0,728495.0,152495.0,744761.0,0.0,0.0,0.0
2,4Q2018,2018,12,51344417.0,17563118.0,2891217.0,459470.0,3201656.0,9167689.0,0.0,1031581.0,0.0,811505.0,6917916.0,0.0,0.0,0.0,0.0,0.0,0.0,3874054.0,0.0,27939.0,1367802.0,15547094.0,9948486.0,0.0,51344417.0,9156188.0,590110.0,4119074.0,765013.0,2473789.0,0.0,0.0,1147626.0,60576.0,0.0,16815756.0,13081776.0,1350.0,1413478.0,118368.0,0.0,2200785.0,0.0,0.0,16027297.0,9345175.0,7960908.0,-69861.0,0.0,728550.0,0.0,725577.0,0.0,0.0,0.0
3,3Q2018,2018,9,56180777.0,22227653.0,2795394.0,679994.0,4193568.0,9124052.0,0.0,941142.0,0.0,4493503.0,6353695.0,0.0,0.0,0.0,0.0,0.0,0.0,3610574.0,0.0,41506.0,1472356.0,15807410.0,10319663.0,0.0,56180777.0,9817638.0,609593.0,4149752.0,614163.0,2841377.0,0.0,0.0,1563664.0,39089.0,0.0,20153991.0,16015383.0,1395.0,1328951.0,266132.0,0.0,2542132.0,0.0,0.0,16571375.0,9637772.0,7956661.0,-69861.0,0.0,174017.0,572546.0,1004408.0,0.0,0.0,0.0
4,2Q2018,2018,6,54517367.0,20704301.0,2638013.0,303724.0,4100884.0,8479855.0,0.0,909635.0,0.0,4272191.0,6252401.0,0.0,0.0,0.0,0.0,0.0,0.0,3511040.0,0.0,42690.0,1454877.0,16037499.0,10068290.0,0.0,54517367.0,9155147.0,509455.0,4204288.0,528291.0,2730270.0,0.0,0.0,1152978.0,29865.0,0.0,20240859.0,16044958.0,1344.0,1608288.0,155660.0,0.0,2430611.0,0.0,0.0,15932185.0,9189175.0,7955263.0,-69861.0,0.0,174017.0,358485.0,771272.0,0.0,0.0,0.0


# DADOS DE PREÇO - API YAHOO FINANCE

Após determinandos os dados fundamentalistas da ação, chegou a hora de extrairmos os dados de preço da API aberta da Yahoo Finance.

Lembrando que temos que convertê-las ao formato de escala tempora de trimestre, para que possamos concatêna-las aos dados fundamentalistas, formando um dataset único da ação. Dados de preço e dados fundamentalistas referentes ao mesmo espaço temporal para cada amostra.

Abaixo, primeiro atribuimos a data de inicio e de fim que coletaremos os dados de preço, e depois trazemos o dataset com base nas datas, e 'm' nos diz respeito a escala temporal mês, como não há trimestre, tivemos que fazer a seguir a manipulação no código, com base nos dados originalmente mensais vindo da API.

In [8]:
####################### COMEÇA O CÓDIGO QUE EXTRAI DADOS DE PREÇO ###############################

start = datetime(2008, 12, 31)
end = datetime(2019, 5, 31)
price_data = web.get_data_yahoo('GOAU4.SA', start, end, interval = 'm')

price_data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-01-01,25.500000,19.75,20.600000,19.950001,21551500.0,17.069397
2009-02-01,23.290001,16.09,19.580000,16.690001,17234800.0,14.280110
2009-03-01,17.900000,14.40,16.299999,16.900000,25269000.0,14.459788
2009-04-01,20.969999,16.41,16.750000,20.520000,21641600.0,17.670584
2009-05-01,27.299999,21.07,21.100000,26.100000,29891100.0,22.475740


As datas na coluna "Date" estavam como índíce, e usamos o code abaixo para tornar "Date" uma coluna normal do dataset, já que vamos precisar manipula-la.

In [9]:
price_data["Date"] = price_data.index
price_data.reset_index(drop=True, inplace=True)


date = price_data['Date']
price_data.drop(labels=['Date'], axis=1,inplace = True)
price_data.insert(0, 'Date', date)

price_data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-01-01,25.500000,19.75,20.600000,19.950001,21551500.0,17.069397
1,2009-02-01,23.290001,16.09,19.580000,16.690001,17234800.0,14.280110
2,2009-03-01,17.900000,14.40,16.299999,16.900000,25269000.0,14.459788
3,2009-04-01,20.969999,16.41,16.750000,20.520000,21641600.0,17.670584
4,2009-05-01,27.299999,21.07,21.100000,26.100000,29891100.0,22.475740


Aqui começa a manipulação para transformarmos os dados de preço de escala mensal para escala trimestral.

Tal qual nos dados fundamentalistas, da coluna Date, criamos as colunas referentes a "dia", "mês" e "ano". Assim a coluna Date não nos é mais útil.

Manipulamos para colocar as três colunas referentes a data nas três primeiras posições das colunas no dataset.

In [10]:
price_data["Day"] = price_data['Date'].map(lambda x: x.day)
price_data["Month"] = price_data['Date'].map(lambda x: x.month)
price_data["Year"] = price_data['Date'].map(lambda x: x.year)

price_data.drop(columns = 'Date', inplace = True)



year = price_data['Year']
price_data.drop(labels=['Year'], axis=1,inplace = True)
price_data.insert(0, 'Year', year)

month = price_data['Month']
price_data.drop(labels=['Month'], axis=1,inplace = True)
price_data.insert(1, 'Month', month)

day = price_data['Day']
price_data.drop(labels=['Day'], axis=1,inplace = True)
price_data.insert(2, 'Day', day)

price_data.head()

,Year,Month,Day,High,Low,Open,Close,Volume,Adj Close
0,2009,1,1,25.500000,19.75,20.600000,19.950001,21551500.0,17.069397
1,2009,2,1,23.290001,16.09,19.580000,16.690001,17234800.0,14.280110
2,2009,3,1,17.900000,14.40,16.299999,16.900000,25269000.0,14.459788
3,2009,4,1,20.969999,16.41,16.750000,20.520000,21641600.0,17.670584
4,2009,5,1,27.299999,21.07,21.100000,26.100000,29891100.0,22.475740


Nessa parte, o "for" percorre todas as linhas do dataset, e tal qual no padrão dos dados fundamentalistas, temos que atribuir aquele padrão que chamei de "Cluster_Month", onde o mês 3 representa o trimestre dos meses 1,2 e 3. Mês 6 o trimestre dos meses 4,5 e 6. E assim por diante.

Após, deletamos a coluna "dia", que não nos é útil.

In [11]:
for i,row in price_data.iterrows():   
    if (row['Month'] == 1) | (row['Month'] == 2) | (row['Month'] == 3):
        price_data.loc[i,'Month'] = 3
    
    elif (row['Month'] == 4) | (row['Month'] == 5) | (row['Month'] == 6):
         price_data.loc[i,'Month'] = 6
        
    elif (row['Month'] == 7) | (row['Month'] == 8) | (row['Month'] == 9):
         price_data.loc[i,'Month'] = 9
        
    else: 
          price_data.loc[i,'Month'] = 12



price_data.drop('Day', axis=1, inplace=True)

price_data.head()

,Year,Month,High,Low,Open,Close,Volume,Adj Close
0,2009,3,25.500000,19.75,20.600000,19.950001,21551500.0,17.069397
1,2009,3,23.290001,16.09,19.580000,16.690001,17234800.0,14.280110
2,2009,3,17.900000,14.40,16.299999,16.900000,25269000.0,14.459788
3,2009,6,20.969999,16.41,16.750000,20.520000,21641600.0,17.670584
4,2009,6,27.299999,21.07,21.100000,26.100000,29891100.0,22.475740


E aqui tempos uma parte bem interessante. Agora agrupamos cada uma das três linhas correspondentes ao trimestre do ano correspondente.

Podemos olhar juntos para a primeira instrução abaixo e para a tabela acima para entender. Agrupamos por Ano e Mês, assim podemos ver as três primeiras linhas com "Month" igual a 3(na verdade são 1,2 e 3) que foram manipulados para facilitar esse processo. Então iremos agrupar essas três linhas, transformando em uma linha só correspondente ao primeiro trimestre de 2014 nesse caso. E assim convertemos os parâmetros de preço conforme suas especificidades.

- Abertura(Open): Utilização do método “first”, atribuindo o primeiro valor do trimestre.
   
- Fechamento(Close): Utilização do método “last”, atribuindo o último valor do trimestre.
      
- Máximo(High):Utilização do método “max”, atribuindo o máximo valor do trimestre.
 
- Mínimo(Low):Utilização do método “min”, atribuindo o mínimo valor do trimestre.

- Volume(Volume): Utilização do método “mean”, atribuindo a média dos valores do trimestre.

- Fechamento Ajustado(Adj Close):Utilização do método “mean”, atribuindo a média dos valores do trimestre.



Depois, novamente o processo de criar uma coluna "Quartes" com uma identificação mais intuitiva do trimestre e ano, e coloca-lo como a primeira posição das colunas.

Além de renomear a coluna "Month" para "Cluster_Month", tudo no mesmo padrão dos dados fundamentalistas.

In [12]:
final_data_price = price_data.groupby(['Year','Month']).agg({'High': 'max', 'Low': 'min', 'Open':'first', 'Close' : 'last', 'Volume':'mean', 'Adj Close':'mean'}).reset_index()


final_data_price['Quarter'] = '1Q2009', '2Q2009', '3Q2009', '4Q2009', '1Q2010', '2Q2010', '3Q2010', '4Q2010', '1Q2011', '2Q2011', '3Q2011', '4Q2011', '1Q2012', '2Q2012', '3Q2012','4Q2012', '1Q2013', '2Q2013', '3Q2013', '4Q2013', '1Q2014', '2Q2014', '3Q2014', '4Q2014', '1Q2015', '2Q2015', '3Q2015', '4Q2015', '1Q2016', '2Q2016', '3Q2016', '4Q2016', '1Q2017', '2Q2017', '3Q2017', '4Q2017', '1Q2018', '2Q2018', '3Q2018', '4Q2018','1Q2019','2Q2019'

quarter = final_data_price['Quarter']
final_data_price.drop(labels=['Quarter'], axis=1, inplace=True)
final_data_price.insert(0, 'Quarter', quarter)

final_data_price.rename(columns={'Month' : 'Cluster_Month'},inplace = True)

final_data_price.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close
0,1Q2009,2009,3,25.500000,14.400000,20.600000,16.900000,2.135177e+07,15.269765
1,2Q2009,2009,6,29.290001,16.410000,16.750000,25.799999,2.616757e+07,20.787909
2,3Q2009,2009,9,31.290001,22.920000,25.969999,29.650000,1.980657e+07,24.246822
3,4Q2009,2009,12,38.099998,28.299999,29.559999,34.919998,2.312820e+07,28.696039
4,1Q2010,2010,3,36.560001,29.350000,35.200001,36.480000,1.651497e+07,28.896974


# UNINDO DADOS DE PREÇO E DADOS FUNDAMENTALISTAS

Simplesmente fazendo um merge entre os dados de preço e fundamentalistas que tratamos até aqui.

A coluna "Year" e "Month" ficará repetida, então precisaremos renomear cada uma e excluir as outras duas que são a mesma coisa.

In [13]:
# MESCLA DOS DADOS DE PREÇO E OS FUNDAMENTALISTAS

dataset = pd.merge(final_data_price, fundamental_data, how='left', on='Quarter')

dataset.rename(columns={'Year_x' : 'Year'},inplace = True)
dataset.rename(columns={'Cluster_Month_x' : 'Cluster_Month'},inplace = True)

dataset.drop('Year_y', axis=1, inplace=True)

dataset.drop('Cluster_Month_y', axis=1, inplace=True)

dataset.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,1Q2009,2009,3,25.500000,14.400000,20.600000,16.900000,2.135177e+07,15.269765,9929192.0,54094.0,138.0,0.0,52559.0,0.0,0.0,0.0,0.0,1397.0,70514.0,0.0,0.0,69872.0,0.0,0.0,0.0,0.0,642.0,0.0,8716353.0,994.0,1087237.0,0.0,9929192.0,13547.0,0.0,3.0,371.0,11139.0,752.0,0.0,1282.0,0.0,0.0,1396928.0,1339043.0,0.0,57524.0,0.0,0.0,361.0,0.0,0.0,0.0,8518717.0,6881998.0,89582.0,0.0,1031137.0,43378.0,472622.0,0.0,0.0,0.0
1,2Q2009,2009,6,29.290001,16.410000,16.750000,25.799999,2.616757e+07,20.787909,9217837.0,22756.0,177.0,0.0,21746.0,0.0,0.0,0.0,0.0,833.0,72607.0,0.0,0.0,72321.0,0.0,0.0,0.0,0.0,0.0,286.0,8034264.0,973.0,1087237.0,0.0,9217837.0,15928.0,0.0,3.0,1431.0,11311.0,1967.0,0.0,1216.0,0.0,0.0,1370124.0,1312137.0,0.0,57623.0,0.0,0.0,364.0,0.0,0.0,0.0,7831785.0,6881998.0,8354.0,0.0,1114672.0,-10915.0,-162324.0,0.0,0.0,0.0
2,3Q2009,2009,9,31.290001,22.920000,25.969999,29.650000,1.980657e+07,24.246822,9235787.0,20794.0,150.0,0.0,19809.0,0.0,0.0,0.0,0.0,835.0,80682.0,0.0,0.0,80519.0,0.0,0.0,0.0,0.0,0.0,163.0,8046123.0,951.0,1087237.0,0.0,9235787.0,18927.0,0.0,2.0,1084.0,11481.0,5325.0,0.0,1035.0,0.0,0.0,1389124.0,1332137.0,0.0,56620.0,0.0,0.0,367.0,0.0,0.0,0.0,7827736.0,6881998.0,9520.0,0.0,1114672.0,265133.0,-443587.0,0.0,0.0,0.0
3,4Q2009,2009,12,38.099998,28.299999,29.559999,34.919998,2.312820e+07,28.696039,9347591.0,213428.0,156.0,0.0,207125.0,0.0,0.0,0.0,0.0,6147.0,84820.0,0.0,0.0,83998.0,0.0,0.0,0.0,0.0,822.0,0.0,7961177.0,929.0,1087237.0,0.0,9347591.0,133001.0,0.0,65.0,21268.0,11653.0,752.0,97427.0,1836.0,0.0,0.0,1356977.0,1306591.0,0.0,50017.0,0.0,0.0,369.0,0.0,0.0,0.0,7857613.0,6881998.0,10632.0,0.0,1434007.0,0.0,-469024.0,0.0,0.0,0.0
4,1Q2010,2010,3,36.560001,29.350000,35.200001,36.480000,1.651497e+07,28.896974,0.0,0.0,4461482.0,0.0,3998456.0,6220345.0,0.0,0.0,0.0,329229.0,3336678.0,0.0,0.0,497063.0,0.0,0.0,0.0,0.0,0.0,2839615.0,0.0,0.0,997882.0,0.0,0.0,4951546.0,0.0,2180336.0,713964.0,1360789.0,0.0,48173.0,648284.0,0.0,0.0,0.0,0.0,0.0,4855763.0,0.0,0.0,575724.0,0.0,0.0,0.0,8622565.0,6881998.0,-69861.0,0.0,104142.0,2282031.0,-575745.0,0.0,0.0,0.0


Ao analisar o dataset, decidi excluir colunas que tinha mais de 1/3 dos dados faltantes, seguindo a lista abaixo de colunas "dropadas".

In [14]:
# exclui colunas que tem mais de 1/3 dos dados como zero ou faltante
dataset = dataset.drop('Ativos Biológicos AC',axis=1)
dataset = dataset.drop('Despesas Antecipadas AC',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas a Valor Justo ACLP',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP',axis=1)
dataset = dataset.drop('Estoques ACLP',axis=1)
dataset = dataset.drop('Ativos Biológicos ACLP',axis=1)
dataset = dataset.drop('Despesas Antecipadas ACLP',axis=1)
dataset = dataset.drop('Créditos com Partes Relacionadas ACLP',axis=1)
dataset = dataset.drop('Diferido',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PC',axis=1)
dataset = dataset.drop('Dividendos e JCP a Pagar PC',axis=1)
dataset = dataset.drop('Provisões PC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PNC',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital PNC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC',axis=1)
dataset = dataset.drop('Lucros e Receitas a Apropriar PNC',axis=1)
dataset = dataset.drop('Reservas de Reavaliação',axis=1)
dataset = dataset.drop('Ajustes de Avaliação Patrimonial',axis=1)
dataset = dataset.drop('Ajustes Acumulados de Conversão',axis=1)
dataset = dataset.drop('Outros Resultados Abrangentes',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital',axis=1)

# APLICAÇÃO DO ALGORTIMO KNN PARA PREENCHIMENTO DOS DADOS FALTANTES

Abaixo, separei apenas as colunas que serão imputadas para treinamento, tirando as de data e os id's. Assim ao invés de um método tradicional como preencher dados faltantes com média ou mediana, utilizei a metodologia com a biblioteca fancy impute, para aplicar KNN e preencher os dados faltantes com esse algortimo.

Na célula abaixo a separação.

Após trocar os valores 0 para "NaN", então realizamos o preenchimento de fato com o KNN.

In [15]:
#separa as colunas que serão usadas para treinamento em 'dataset'
pricing = dataset.iloc[0:,0:3]
dataset = dataset.iloc[0:,3:]
dataset.to_numpy()

X = dataset.replace(0, np.nan, inplace=True)
dataset.replace(0, np.nan, inplace=True)
final_data = KNN(k=3).fit_transform(dataset)

# ela tira as labels das colunas e aqui as coloco de volta
dataset = pd.DataFrame(data = final_data, columns = ['High','Low','Open','Close','Volume','Adj Close','Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC','Tributos a Recuperar AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo','Contas a Receber ACLP','Tributos Diferidos ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Outros PC','Passivo Não Circulante','Empréstimos e Financiamentos PNC','Outros PNC','Tributos Diferidos PNC','Provisões PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Lucros','Lucros/Prejuízos Acumulados'])
dataset = pd.concat([pricing, dataset], axis=1)

inference = dataset.iloc[-1,0:].to_frame().T
inference

Imputing row 1/42 with 8 missing, elapsed time: 0.004


,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,2Q2019,2019,6,7.7,6.34,7.22,6.48,1.84723e+08,6.52167,5.11707e+07,1.69376e+07,1.75095e+06,390607,3.73242e+06,9.53191e+06,960663,571076,6.53098e+06,208509,3.92956e+06,63126,1.69537e+06,1.62583e+07,9.74838e+06,5.11707e+07,7.96492e+06,482729,3.83283e+06,546534,2.12276e+06,919099,1.68249e+07,1.28e+07,1.93929e+06,61015,2.01944e+06,1.68164e+07,9.56446e+06,7.96551e+06,-69861,728495,245699


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 42 columns):
Quarter                                          42 non-null object
Year                                             42 non-null int64
Cluster_Month                                    42 non-null int64
High                                             42 non-null float64
Low                                              42 non-null float64
Open                                             42 non-null float64
Close                                            42 non-null float64
Volume                                           42 non-null float64
Adj Close                                        42 non-null float64
Ativo Total                                      42 non-null float64
Ativo Circulante                                 42 non-null float64
Caixa e Equivalentes de Caixa AC                 42 non-null float64
Aplicações Financeiras AC                        42 non-null float64
Conta

# AQUI FINALMENTE PRODUZIMO A NOSSA AMOSTRA QUE SERÁ IMPUTADA AO NOSSO MODELO, SIMULANDO UMA SITUAÇÃO REAL. AO IMPUTAR OS DADOS DE PREÇO E DE FUNDAMENTOS CONFORME OS USADOS NO MODELO, QUAL STATUS DO TARGET ELE RETORNARÁ FRENTE A UMA POSSIVEL VALORIZAÇÃO ?

In [17]:
inference = inference.iloc[0:,3:]
inference

,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,7.7,6.34,7.22,6.48,1.84723e+08,6.52167,5.11707e+07,1.69376e+07,1.75095e+06,390607,3.73242e+06,9.53191e+06,960663,571076,6.53098e+06,208509,3.92956e+06,63126,1.69537e+06,1.62583e+07,9.74838e+06,5.11707e+07,7.96492e+06,482729,3.83283e+06,546534,2.12276e+06,919099,1.68249e+07,1.28e+07,1.93929e+06,61015,2.01944e+06,1.68164e+07,9.56446e+06,7.96551e+06,-69861,728495,245699


In [18]:
inference.to_csv("ROW_INFERENCE_GOAU4.csv", index= False)